In [1]:
import pandas as pd
import numpy as np
import time 
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder # to slace the dataset
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV # to cross validate accuracy
from gen_ball import gen_balls
from classGBTSVM import OvO_GBTSVM, GBTSVM
from classLGBTSVM import LGBTSVM, OvO_LGBTSVM, OvR_LGBTSVM

In [2]:
directory = r"D:/LGBTSVM_MCD/LGBTSVM-MCD"
file_path = os.path.join(directory, "winequality-white_new.csv")
file_data = np.loadtxt(file_path, delimiter=',')
labels = file_data[:, -1]

# Lọc dữ liệu chỉ lấy các hàng có nhãn là 5, 6, hoặc 7
filtered_data = file_data[np.isin(labels, [5, 6])]
m, n = filtered_data.shape
for i in range(m):
    if filtered_data[i, n-1] == 5:
        filtered_data[i, n-1] = -1
    else:
        filtered_data[i, n-1] = 1
print("Kích thước dữ liệu sau khi lọc:", filtered_data.shape)

Kích thước dữ liệu sau khi lọc: (3655, 12)


In [3]:
def filter_outliers(data, factor=1.5):
    """
    Loại bỏ các hàng có giá trị ngoại lai (outliers) dựa trên IQR cho các thuộc tính.
    
    Parameters:
        data (numpy.ndarray): Ma trận dữ liệu với các cột thuộc tính và cột nhãn ở cuối.
        factor (float): Hệ số nhân cho IQR. Giá trị mặc định là 1.5.
        
    Returns:
        numpy.ndarray: Dữ liệu sau khi loại bỏ các outlier.
    """
    # Chọn các cột thuộc tính (loại bỏ cột nhãn cuối cùng)
    features = data[:, :-1]
    
    # Tính phần tư thứ nhất (Q1) và phần tư thứ ba (Q3)
    q1 = np.percentile(features, 25, axis=0)
    q3 = np.percentile(features, 75, axis=0)
    
    # Tính IQR cho mỗi cột thuộc tính
    iqr = q3 - q1
    
    # Xác định biên dưới và biên trên cho mỗi cột
    lower_bound = q1 - factor * iqr
    upper_bound = q3 + factor * iqr
    
    # Tạo mặt nạ boolean: chỉ giữ các hàng mà mọi cột thuộc tính đều nằm trong khoảng cho phép
    mask = np.all((features >= lower_bound) & (features <= upper_bound), axis=1)
    return data[mask]

# Lọc bỏ các outlier từ dữ liệu đã lọc nhãn
filtered_data_no_outliers = filter_outliers(filtered_data, factor=1.5)
print("Kích thước dữ liệu sau khi lọc outlier:", filtered_data_no_outliers.shape)

Kích thước dữ liệu sau khi lọc outlier: (2989, 12)


In [4]:
scaler = MinMaxScaler(feature_range=(-1, 1))
filtered_data_no_outliers[:, :-1] = scaler.fit_transform(filtered_data_no_outliers[:, :-1])
filtered_data_no_outliers[:, :-1] = np.round(filtered_data_no_outliers[:, :-1], 5)

In [5]:
labels = filtered_data_no_outliers[:, -1]

# Lấy index (chỉ số) của các mẫu nhãn 5
# indices_label_5 = np.where(labels == 5)[0]
# remove_5 = np.random.choice(indices_label_5, size=300, replace=False)

# Tương tự với nhãn 6
indices_label_6 = np.where(labels == 1)[0]
remove_6 = np.random.choice(indices_label_6, size=700, replace=False)

# Gộp hai mảng chỉ số cần xóa
# indices_to_remove = np.concatenate((remove_5, remove_6))

# Xóa các dòng tương ứng khỏi filtered_data
filtered_data_reduced = np.delete(filtered_data_no_outliers, remove_6, axis=0)
m, n = filtered_data_reduced.shape
# Kiểm tra kích thước sau khi giảm bớt mẫu
print("Kích thước dữ liệu sau khi giảm mẫu:", filtered_data_reduced.shape)

Kích thước dữ liệu sau khi giảm mẫu: (2289, 12)


In [6]:
filtered_data_reduced

array([[-0.25581,  0.08642,  0.03571, ...,  0.04   , -0.55102,  1.     ],
       [ 0.16279, -0.25926, -0.03571, ..., -0.32   , -0.38776,  1.     ],
       [ 0.16279, -0.25926, -0.03571, ..., -0.32   , -0.38776,  1.     ],
       ...,
       [-0.11628,  0.18519,  0.10714, ..., -0.08   , -0.5102 , -1.     ],
       [-0.16279, -0.20988, -0.5    , ..., -0.08   , -0.59184,  1.     ],
       [-0.39535, -0.35802,  0.17857, ..., -0.64   ,  0.38776,  1.     ]])

In [7]:
np.random.seed(0)
indices = np.random.permutation(m)
filtered_data_reduced = filtered_data_reduced[indices]
A_train=filtered_data_reduced[0:int(m*(1-0.20))]
A_test=filtered_data_reduced[int(m * (1-0.20)):]

In [8]:
pur = 1 - (0.015 * 5)          
# pur = 1 - (0.015 * 10)                
num = 4
c1=0.00001
c2=0.00001
            
A_train = gen_balls(A_train, pur=pur, delbals=num)

Radius=[]
for i in A_train:
    Radius.append(i[1])
Center=[]
for ii in A_train:
    Center.append(ii[0])
Label=[]
for iii in A_train:
    Label.append(iii[-1])
Radius=np.array(Radius)
Center=np.array(Center)
Label=np.array(Label)
Z_train=np.hstack((Center,Radius.reshape(Radius.shape[0], 1)))
Lab=Label.reshape(Label.shape[0], 1)
A_train=np.hstack((Z_train,Lab))

In [9]:
X_train = A_train[:,:-1]
X_test = A_test[:,:-1]
y_train = A_train[:,-1]
y_test = A_test[:,-1]

In [10]:
ovo_tsvm = OvO_GBTSVM(d1=0.1, d2=0.1, eps1=0.05, eps2=0.05)

# Huấn luyện mô hình
ovo_tsvm.fit(X_train, y_train)
y_pred = ovo_tsvm.predict(X_test)
print(y_test)
# Đánh giá độ chính xác
accuracy = ovo_tsvm.score(X_test, y_test)

accuracy

[ 1.  1.  1. -1.  1.  1. -1.  1. -1.  1. -1.  1.  1. -1.  1. -1. -1. -1.
 -1.  1. -1. -1.  1.  1.  1. -1. -1.  1. -1. -1. -1.  1.  1.  1.  1. -1.
 -1. -1. -1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.
 -1.  1.  1. -1.  1.  1.  1.  1.  1. -1.  1. -1.  1. -1.  1.  1. -1. -1.
  1. -1. -1.  1.  1.  1.  1.  1. -1. -1. -1.  1.  1.  1.  1. -1. -1. -1.
 -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1. -1.  1. -1. -1.
  1.  1. -1.  1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.
  1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1. -1. -1.  1.  1. -1.  1. -1.
  1. -1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1.  1. -1.  1. -1.  1. -1.
  1. -1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1. -1.  1. -1. -1. -1. -1.
  1. -1.  1.  1. -1. -1.  1.  1. -1. -1. -1.  1.  1.  1.  1. -1.  1.  1.
  1.  1. -1. -1.  1.  1.  1. -1. -1. -1. -1.  1. -1.  1.  1. -1.  1.  1.
  1.  1.  1. -1. -1. -1. -1.  1. -1. -1.  1.  1.  1.  1.  1.  1. -1.  1.
  1.  1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1.  1

67.24890829694323

In [11]:
ovo_tsvm = OvO_LGBTSVM(d1=0.1, d2=0.1, d3=0.1, d4=0.1)

# Huấn luyện mô hình
ovo_tsvm.fit(X_train, y_train)
y_pred = ovo_tsvm.predict(X_test)
print(y_test)
# Đánh giá độ chính xác
accuracy = ovo_tsvm.score(X_test, y_test)

accuracy

[ 1.  1.  1. -1.  1.  1. -1.  1. -1.  1. -1.  1.  1. -1.  1. -1. -1. -1.
 -1.  1. -1. -1.  1.  1.  1. -1. -1.  1. -1. -1. -1.  1.  1.  1.  1. -1.
 -1. -1. -1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.
 -1.  1.  1. -1.  1.  1.  1.  1.  1. -1.  1. -1.  1. -1.  1.  1. -1. -1.
  1. -1. -1.  1.  1.  1.  1.  1. -1. -1. -1.  1.  1.  1.  1. -1. -1. -1.
 -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1. -1.  1. -1. -1.
  1.  1. -1.  1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.
  1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1. -1. -1.  1.  1. -1.  1. -1.
  1. -1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1.  1. -1.  1. -1.  1. -1.
  1. -1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1. -1.  1. -1. -1. -1. -1.
  1. -1.  1.  1. -1. -1.  1.  1. -1. -1. -1.  1.  1.  1.  1. -1.  1.  1.
  1.  1. -1. -1.  1.  1.  1. -1. -1. -1. -1.  1. -1.  1.  1. -1.  1.  1.
  1.  1.  1. -1. -1. -1. -1.  1. -1. -1.  1.  1.  1.  1.  1.  1. -1.  1.
  1.  1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1.  1

0.5458515283842795

In [12]:
ovo_tsvm = OvR_LGBTSVM(d1=0.1, d2=0.1, d3=0.1, d4=0.1)

# Huấn luyện mô hình
ovo_tsvm.fit(X_train, y_train)
y_pred = ovo_tsvm.predict(X_test)
print(y_test)
# Đánh giá độ chính xác
accuracy = ovo_tsvm.score(X_test, y_test)

accuracy

[ 1.  1.  1. -1.  1.  1. -1.  1. -1.  1. -1.  1.  1. -1.  1. -1. -1. -1.
 -1.  1. -1. -1.  1.  1.  1. -1. -1.  1. -1. -1. -1.  1.  1.  1.  1. -1.
 -1. -1. -1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.
 -1.  1.  1. -1.  1.  1.  1.  1.  1. -1.  1. -1.  1. -1.  1.  1. -1. -1.
  1. -1. -1.  1.  1.  1.  1.  1. -1. -1. -1.  1.  1.  1.  1. -1. -1. -1.
 -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1. -1.  1. -1. -1.
  1.  1. -1.  1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.
  1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1. -1. -1.  1.  1. -1.  1. -1.
  1. -1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1.  1. -1.  1. -1.  1. -1.
  1. -1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1. -1.  1. -1. -1. -1. -1.
  1. -1.  1.  1. -1. -1.  1.  1. -1. -1. -1.  1.  1.  1.  1. -1.  1.  1.
  1.  1. -1. -1.  1.  1.  1. -1. -1. -1. -1.  1. -1.  1.  1. -1.  1.  1.
  1.  1.  1. -1. -1. -1. -1.  1. -1. -1.  1.  1.  1.  1.  1.  1. -1.  1.
  1.  1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1.  1

0.5458515283842795